In [5]:
import glob 
import os
import numpy as np
import nibabel as nb
from nilearn import image
from joblib import Parallel, delayed

In [6]:
# setup dirs
home_dir = '/home/shared/2018/visual/cerebellum_prf/'
der_dir = os.path.join(home_dir,'derivatives')
out_dir = os.path.join(home_dir,'derivatives','pp')
tsnrt_out_dir = os.path.join(out_dir,'tsnr')
stand_out_dir = os.path.join(out_dir,'stand')

subs = ['02','03']#,'02','03']
# sess = {
#     '01':['01','03','02'],
#     '02':['01','02','03','04'],
#     '03':['01','02','03']
# }
sess_combs = {
    '01':[['01','02','03']],
    '02':[['01','02','03','04']],
    '03':[['01','02','03']],#['01','02','03']
}

TR = 1.5
space = 'MNI152NLin2009cAsym' # 'T1w'


# in_postFix = '_resampled_fnirted_smoothed_sgtf_ng'
in_postFix = '_resampled_fnirted_smoothed_sgtf'

dtype = 'zscore' # or psc
stand_out_dir = os.path.join(out_dir,dtype)
if not os.path.isdir(stand_out_dir): os.mkdir(stand_out_dir)

n_folds = 10
proportion_train = 0.75

In [7]:
def load_data(fn):
    img = nb.load(fn)
    data = img.get_data()

    return data
    

In [8]:
# # create avg files
for sub in subs:
    for sess in sess_combs[sub]:
        all_sess_avg_data = []
        all_sess_tsnr_data = []
        for ses in sess:

            # first collect all data
            sj_stand_out_dir = os.path.join(stand_out_dir,'sub-%s'%sub)
            fns = sorted(glob.glob(os.path.join(sj_stand_out_dir,'sub-%s_ses-%s*bold_space-%s_preproc%s_zscore.nii.gz'%(sub,ses,space,in_postFix))))

            all_data = Parallel(n_jobs=len(fns),verbose=9)(delayed(load_data)(fn)  for fn in fns)

            all_sess_avg_data.extend(all_data)
            sample_img = nb.load(fns[0])
            # now get tsnrs for weighted avg
            sj_tsnr_out_dir = os.path.join(out_dir,'tsnr','sub-%s'%sub)
            fns = sorted(glob.glob(os.path.join(sj_tsnr_out_dir,'sub-%s_ses-%s*bold_space-%s_preproc%s_tsnr.nii.gz'%(sub,ses,space,in_postFix))))

            all_tsnrs = Parallel(n_jobs=len(fns),verbose=9)(delayed(load_data)(fn)  for fn in fns)

            # make sure tsnrs run from 0.01 to maxval
            all_tsnrs = np.array(all_tsnrs)
            all_tsnrs[np.isnan(all_tsnrs)] = 0
            all_tsnrs[np.isinf(all_tsnrs)] = 0
            all_tsnrs += np.abs(np.min(all_tsnrs))+0.01

            all_sess_tsnr_data.extend(all_tsnrs)
            
        all_sess_avg_data = np.nan_to_num(all_sess_avg_data)
        all_sess_tsnr_data = np.array(all_sess_tsnr_data)
        
        print 'now computing tsnr weighted mean over runs across sessions'
        avg_over_runs = np.average(all_sess_avg_data,weights=np.tile(all_sess_tsnr_data[:,:,:,:,np.newaxis],(1,1,1,1,120)),axis=0)
        new_img = nb.Nifti1Image(avg_over_runs,affine=sample_img.affine,header=sample_img.header)
        out_fn = os.path.join(sj_stand_out_dir,'tsnr_weighted_mean_of%s_over_runs_ses_%s.nii.gz'%(in_postFix,''.join(sess)))
        nb.save(new_img,out_fn)

        # now create test and train sets
        n_runs = np.shape(all_sess_tsnr_data)[0]
        n_train = np.ceil(n_runs*proportion_train)

        for k in range(n_folds):
            t  = np.arange(n_runs)
            np.random.shuffle(t)
            train_runs = t[:n_train]
            test_runs = t[n_train:]
            
            print 'now computing tsnr weighted mean over runs across sessions, train set fold %d'%k
            avg_over_runs = np.average(all_sess_avg_data[train_runs],weights=np.tile(all_sess_tsnr_data[train_runs,:,:,:,np.newaxis],(1,1,1,1,120)),axis=0)
            new_img = nb.Nifti1Image(avg_over_runs,affine=sample_img.affine,header=sample_img.header)
            out_fn = os.path.join(sj_stand_out_dir,'tsnr_weighted_mean_of%s_over_runs_ses_%s_train_%d.nii.gz'%(in_postFix,''.join(sess),k))
            nb.save(new_img,out_fn)
            
            print 'now computing tsnr weighted mean over runs across sessions, test set fold %d'%k
            avg_over_runs = np.average(all_sess_avg_data[test_runs],weights=np.tile(all_sess_tsnr_data[test_runs,:,:,:,np.newaxis],(1,1,1,1,120)),axis=0)
            new_img = nb.Nifti1Image(avg_over_runs,affine=sample_img.affine,header=sample_img.header)
            out_fn = os.path.join(sj_stand_out_dir,'tsnr_weighted_mean_of%s_over_runs_ses_%s_test_%d.nii.gz'%(in_postFix,''.join(sess),k))
            nb.save(new_img,out_fn)


[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   2 out of  13 | elapsed:   24.8s remaining:  2.3min
[Parallel(n_jobs=13)]: Done   4 out of  13 | elapsed:   27.8s remaining:  1.0min
[Parallel(n_jobs=13)]: Done   6 out of  13 | elapsed:   30.9s remaining:   36.0s
[Parallel(n_jobs=13)]: Done   8 out of  13 | elapsed:   34.1s remaining:   21.3s
[Parallel(n_jobs=13)]: Done  10 out of  13 | elapsed:   37.4s remaining:   11.2s
[Parallel(n_jobs=13)]: Done  13 out of  13 | elapsed:   42.5s finished
[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   2 out of  13 | elapsed:    0.2s remaining:    0.9s
[Parallel(n_jobs=13)]: Done   4 out of  13 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=13)]: Done   6 out of  13 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=13)]: Done   8 out of  13 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=13)]: Done  10 out of  13 | e

now computing tsnr weighted mean over runs across sessions


/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/numpy/lib/function_base.py:961: RuntimeWarning: overflow encountered in multiply
  avg = np.multiply(a, wgt).sum(axis)/scl
/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/ipykernel/__main__.py:46: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


now computing tsnr weighted mean over runs across sessions, train set fold 0
now computing tsnr weighted mean over runs across sessions, test set fold 0
now computing tsnr weighted mean over runs across sessions, train set fold 1
now computing tsnr weighted mean over runs across sessions, test set fold 1
now computing tsnr weighted mean over runs across sessions, train set fold 2
now computing tsnr weighted mean over runs across sessions, test set fold 2
now computing tsnr weighted mean over runs across sessions, train set fold 3
now computing tsnr weighted mean over runs across sessions, test set fold 3
now computing tsnr weighted mean over runs across sessions, train set fold 4
now computing tsnr weighted mean over runs across sessions, test set fold 4
now computing tsnr weighted mean over runs across sessions, train set fold 5
now computing tsnr weighted mean over runs across sessions, test set fold 5
now computing tsnr weighted mean over runs across sessions, train set fold 6
now c

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:   29.5s remaining:  1.5min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:   32.4s remaining:   45.4s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:   35.5s remaining:   25.3s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:   38.3s remaining:   12.8s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:   42.7s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:    0.5s remaining:    1.6s
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:    0.6s remaining:    0.8s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:    0.6s remaining:    0.4s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:    0.6s remaining:    0.2s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.6s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12

now computing tsnr weighted mean over runs across sessions
now computing tsnr weighted mean over runs across sessions, train set fold 0
now computing tsnr weighted mean over runs across sessions, test set fold 0
now computing tsnr weighted mean over runs across sessions, train set fold 1
now computing tsnr weighted mean over runs across sessions, test set fold 1
now computing tsnr weighted mean over runs across sessions, train set fold 2
now computing tsnr weighted mean over runs across sessions, test set fold 2
now computing tsnr weighted mean over runs across sessions, train set fold 3
now computing tsnr weighted mean over runs across sessions, test set fold 3
now computing tsnr weighted mean over runs across sessions, train set fold 4
now computing tsnr weighted mean over runs across sessions, test set fold 4
now computing tsnr weighted mean over runs across sessions, train set fold 5
now computing tsnr weighted mean over runs across sessions, test set fold 5
now computing tsnr weig